# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [2]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [3]:
step3_output = pd.read_csv(r"C:/Users/Curtis/Documents/SpringboardGuidedCapstone/data/step3_output.csv")
step3_output.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,cluster
0,Hilltop Ski Area,Alaska,2090,294,0,0,0,0,0,1,...,30,30,150,36,69,30,34,152,30,0
1,Sunrise Park Resort,Arizona,11100,1800,0,0,0,1,2,3,...,800,80,115,49,250,74,78,104,80,1
2,Yosemite Ski & Snowboard Area,California,7800,600,0,0,0,0,0,1,...,88,174,110,84,300,47,47,107,0,1
3,Boreal Mountain Resort,California,7700,500,0,0,0,1,1,3,...,380,200,150,54,400,49,64,150,200,1
4,Dodge Ridge,California,8200,1600,0,0,0,0,1,2,...,862,174,115,69,350,78,78,140,0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [4]:
print(step3_output.head())

                            Name       state  summit_elev  vertical_drop  \
0               Hilltop Ski Area      Alaska         2090            294   
1            Sunrise Park Resort     Arizona        11100           1800   
2  Yosemite Ski & Snowboard Area  California         7800            600   
3         Boreal Mountain Resort  California         7700            500   
4                    Dodge Ridge  California         8200           1600   

   trams  fastEight  fastSixes  fastQuads  quad  triple  ...  \
0      0          0          0          0     0       1  ...   
1      0          0          0          1     2       3  ...   
2      0          0          0          0     0       1  ...   
3      0          0          0          1     1       3  ...   
4      0          0          0          0     1       2  ...   

   SkiableTerrain_ac  Snow Making_ac  daysOpenLastYear  yearsOpen  \
0                 30              30               150         36   
1                800

In [5]:
#dfo = pd.get_dummies(step3_output['state']) # select object type columns
dfo = pd.get_dummies(step3_output, prefix=['state'], columns=['state'])

In [6]:
print(dfo.head())

                            Name  summit_elev  vertical_drop  trams  \
0               Hilltop Ski Area         2090            294      0   
1            Sunrise Park Resort        11100           1800      0   
2  Yosemite Ski & Snowboard Area         7800            600      0   
3         Boreal Mountain Resort         7700            500      0   
4                    Dodge Ridge         8200           1600      0   

   fastEight  fastSixes  fastQuads  quad  triple  double  ...  \
0          0          0          0     0       1       0  ...   
1          0          0          1     2       3       1  ...   
2          0          0          0     0       1       3  ...   
3          0          0          1     1       3       1  ...   
4          0          0          0     1       2       5  ...   

   state_Rhode Island  state_South Dakota  state_Tennessee  state_Utah  \
0                   0                   0                0           0   
1                   0             

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [7]:
# first we import the preprocessing package from the sklearn library
from sklearn.preprocessing import StandardScaler

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = dfo.drop(columns=['Name','AdultWeekend'], axis=0)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y= dfo['AdultWeekend']
# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [8]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [9]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [10]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [11]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [12]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error

In [13]:
explained_variance_score(y_test, y_pred)

0.5891944573538532

In [14]:
mean_absolute_error(y_test, y_pred)

6.574602599373279

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [15]:
lm.intercept_

18.92895416339629

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [16]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by='Coefficient', ascending=False)

,Coefficient
state_Tennessee,1.962159e+01
state_Virginia,1.617539e+01
state_Rhode Island,1.225924e+01
state_North Carolina,1.215140e+01
state_West Virginia,1.132916e+01
state_Massachusetts,1.074576e+01
state_Idaho,1.046688e+01
state_South Dakota,9.882582e+00
state_Montana,9.718797e+00
state_Arizona,9.438530e+00


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [17]:
# first we import the preprocessing package from the sklearn library
from sklearn.preprocessing import StandardScaler

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = step3_output.drop(columns=['Name','AdultWeekend', 'state'], axis=0)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y= step3_output['AdultWeekend']
# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [18]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [19]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

In [20]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

In [21]:
explained_variance_score(y_test, y_pred)

0.5245938783397672

In [22]:
mean_absolute_error(y_test, y_pred)

7.3744015559107

In [23]:
lm.intercept_

19.29175918166346

In [24]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by='Coefficient', ascending=False)

,Coefficient
cluster,4.846554e+00
quad,2.185368e+00
TerrainParks,1.770193e+00
surface,1.447266e+00
LongestRun_mi,1.419768e+00
total_chairs,7.267223e-01
AdultWeekday,6.438278e-01
fastQuads,4.408871e-01
double,2.274581e-01
triple,2.248085e-01


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [28]:
# first we import the preprocessing package from the sklearn library
from sklearn.preprocessing import StandardScaler

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = step3_output.drop(columns=['Name','AdultWeekend', 'state', 'summit_elev'], axis=0)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y= step3_output['AdultWeekend']
# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [29]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [30]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

In [31]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

In [32]:
explained_variance_score(y_test, y_pred)

0.5148827252793

In [33]:
mean_absolute_error(y_test, y_pred)

7.615394391127729

In [34]:
lm.intercept_

18.904963574391836

In [27]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by='Coefficient', ascending=False)

,Coefficient
cluster,4.846554e+00
quad,2.185368e+00
TerrainParks,1.770193e+00
surface,1.447266e+00
LongestRun_mi,1.419768e+00
total_chairs,7.267223e-01
AdultWeekday,6.438278e-01
fastQuads,4.408871e-01
double,2.274581e-01
triple,2.248085e-01


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection: